In [1]:
import glob, os


In [3]:
# I used below lines to generate the stop_words
#from nltk.corpus import stopwords
#sw = stopwords.words("english")
sw=['i', 'me', 'my', 'myself', 'we', 'our', 'ours', 'ourselves', 'you', "you're", "you've", "you'll", "you'd", 'your', 'yours', 'yourself', 'yourselves', 'he', 'him', 'his', 'himself', 'she', "she's", 'her', 'hers', 'herself', 'it', "it's", 'its', 'itself', 'they', 'them', 'their', 'theirs', 'themselves', 'what', 'which', 'who', 'whom', 'this', 'that', "that'll", 'these', 'those', 'am', 'is', 'are', 'was', 'were', 'be', 'been', 'being', 'have', 'has', 'had', 'having', 'do', 'does', 'did', 'doing', 'a', 'an', 'the', 'and', 'but', 'if', 'or', 'because', 'as', 'until', 'while', 'of', 'at', 'by', 'for', 'with', 'about', 'against', 'between', 'into', 'through', 'during', 'before', 'after', 'above', 'below', 'to', 'from', 'up', 'down', 'in', 'out', 'on', 'off', 'over', 'under', 'again', 'further', 'then', 'once', 'here', 'there', 'when', 'where', 'why', 'how', 'all', 'any', 'both', 'each', 'few', 'more', 'most', 'other', 'some', 'such', 'no', 'nor', 'not', 'only', 'own', 'same', 'so', 'than', 'too', 'very', 's', 't', 'can', 'will', 'just', 'don', "don't", 'should', "should've", 'now', 'd', 'll', 'm', 'o', 're', 've', 'y', 'ain', 'aren', "aren't", 'couldn', "couldn't", 'didn', "didn't", 'doesn', "doesn't", 'hadn', "hadn't", 'hasn', "hasn't", 'haven', "haven't", 'isn', "isn't", 'ma', 'mightn', "mightn't", 'mustn', "mustn't", 'needn', "needn't", 'shan', "shan't", 'shouldn', "shouldn't", 'wasn', "wasn't", 'weren', "weren't", 'won', "won't", 'wouldn', "wouldn't"]



In [4]:
#to find the actual words
import nltk 
import re
from nltk.tokenize import word_tokenize 

In [5]:
#updates the dictionary for a single file
def updateDictionary(file,words_dictionary):

    file = open(file,newline='')
    result = file.read()
    words = word_tokenize(result)
    for line in words:
        # used regex to avoid invalid sequence of characters as english words
        r1 = re.findall(r"\b([a-zA-Z]+)\b",line)
        
        for w in r1:
                if (w.lower() in sw)==False:
                    if w.lower() in words_dictionary:
                        words_dictionary[w.lower()]=words_dictionary[w.lower()]+1
                    else:
                        words_dictionary[w.lower()]=1
    
    return words_dictionary

In [6]:
Y_data=[0 for i in range(0,19997)] #initialized Y_data

In [7]:
#returns list of files and labelled Y_data as 1 to 20 for each folder

def getListOfFiles(dirName,cnt,Y_data):
    # create a list of file and sub directories 
    # names in the given directory 
  
    listOfFile = os.listdir(dirName)
    allFiles = list()
    # Iterate over all the entries
    for entry in listOfFile:
        # Create full path
        fullPath = os.path.join(dirName, entry)
        # If entry is a directory then get the list of files in this directory 
        if os.path.isdir(fullPath):
            cnt=cnt+1
            files_in_folder,Y_data=getListOfFiles(fullPath,cnt,Y_data)
           
            start=len(allFiles)
            end=start+len(files_in_folder)
           
            for i in range(start,end):
                Y_data[i]=cnt
            allFiles = allFiles + files_in_folder
            
        else:
            allFiles.append(fullPath)
            
    return allFiles,Y_data

In [8]:
allFilesNames,Y_data=getListOfFiles('F:\\ML\\20_newsgroups(2)\\20_newsgroups',0,Y_data)


In [9]:
def returnDictionary():
    
    words_dictionary={}
    for file in allFilesNames:
        words_dictionary=updateDictionary(file,words_dictionary)
    return words_dictionary

In [10]:
#initial dictionary
complete_dict=returnDictionary()
print(len(complete_dict))

106756


In [11]:
#sorting the dictionary using values
import operator
sorted_result = sorted(complete_dict.items(), key=operator.itemgetter(1))
#pick top 3000 words

sorted_result=sorted_result[-3001:-1]
#sorted_result is a list of tuples, converted to dictionary
dictionary = {k:v for k,v in sorted_result}


In [13]:
# X= final input data

X=[]

#filling the values of Y
Y=Y_data

#constructing the dataset going through each and every file
for file in allFilesNames:
    file = open(file,newline='')
    result = file.read()
    words = word_tokenize(result)
    x_row_file=[0 for i in range(0,3000)]
    for line in words:

        r1 = re.findall(r"\b([a-zA-Z]+)\b",line)
        
        for w in r1:
                if not(w.lower() in dictionary):
                    continue
                else:
                    x_row_file[list(dictionary).index(w.lower())]=x_row_file[list(dictionary).index(w.lower())]+1
    X.append(x_row_file)
        

19997


In [14]:
from sklearn import model_selection

X_train,X_test,Y_train,Y_test = model_selection.train_test_split(X,Y,test_size=0.25,random_state=0)

3000


In [15]:
from sklearn.metrics import classification_report, confusion_matrix

In [16]:
#using sklearn
from sklearn.naive_bayes import MultinomialNB
clf=MultinomialNB()
clf.fit(X_train,Y_train)
Y_pred=clf.predict(X_test)
print(classification_report(Y_test,Y_pred))
print(confusion_matrix(Y_test,Y_pred))

             precision    recall  f1-score   support

          1       0.81      0.83      0.82       233
          2       0.70      0.81      0.75       253
          3       0.72      0.05      0.10       249
          4       0.50      0.88      0.63       240
          5       0.76      0.87      0.81       236
          6       0.83      0.75      0.79       240
          7       0.83      0.90      0.86       261
          8       0.88      0.89      0.89       269
          9       0.90      0.97      0.93       284
         10       0.95      0.93      0.94       248
         11       0.94      0.98      0.96       231
         12       0.97      0.96      0.96       233
         13       0.86      0.86      0.86       244
         14       0.95      0.86      0.90       256
         15       0.96      0.93      0.94       246
         16       0.96      1.00      0.98       252
         17       0.82      0.91      0.86       249
         18       0.93      0.87      0.90   

[[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 2, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 3, 2, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 2, 1, 0, 0, 0, 0, 0, 0, 0, 0, 4, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 2, 2, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 5, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 4, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 4, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 2, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 2, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 2, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,

In [37]:
#implementing Naive Classifier
import numpy as np

In [38]:
def fit(X_train_in, Y_train_in):
    result = {}
    X_train=np.array(X_train_in)
    Y_train=np.array(Y_train_in)
    print("Y train is",Y_train)
    class_values = set(Y_train)
    for current_class in class_values:
        result[current_class] = {}
        
        result["total_data"] = len(Y_train)
        current_class_rows = (Y_train == current_class)
        X_train_current = X_train[current_class_rows]
        Y_train_current = Y_train[current_class_rows]
        num_features = len(X_train[0])
        result[current_class]["total_count"] = len(Y_train_current)
        for j in range(1, num_features + 1):
            result[current_class][j] = {}
            all_possible_values = set(X_train[:, j - 1])
            for current_value in all_possible_values:
                result[current_class][j][current_value] = (X_train_current[:, j - 1] == current_value).sum()
                
    return result

In [47]:
def probability(dictionary, x, current_class):
    output = np.log(dictionary[current_class]["total_count"]) - np.log(dictionary["total_data"])
    num_features = len(dictionary[current_class].keys()) - 1;
    for j in range(1, num_features + 1):
        xj = x[j - 1]
        count_current_class_with_value_xj=1
        if  xj in dictionary[current_class][j]:
            count_current_class_with_value_xj = dictionary[current_class][j][xj] + 1
        count_current_class = dictionary[current_class]["total_count"] + len(dictionary[current_class][j].keys())
        current_xj_probablity = np.log(count_current_class_with_value_xj) - np.log(count_current_class)
        output = output + current_xj_probablity
    return output

In [48]:
def predictSinglePoint(dictionary, x):
    classes = dictionary.keys()
    best_p = -1000
    best_class = -1
    first_run = True
    for current_class in classes:
        if (current_class == "total_data"):
            continue
        p_current_class = probability(dictionary, x, current_class)
        if (first_run or p_current_class > best_p):
            best_p = p_current_class
            best_class = current_class
        first_run = False
    return best_class

In [49]:
def predict(dictionary, X_test):
    y_pred = []
    for x in X_test:
        x_class = predictSinglePoint(dictionary, x)
        y_pred.append(x_class)
    return y_pred

3000


In [51]:
#dictionary2 is for my naive classifier,which is multi-level
dictionary2 = fit(X_train,Y_train)


Y train is [19 20 13 ... 10 11  3]


In [52]:
Y_pred = predict(dictionary2,X_test)

In [53]:
from sklearn.metrics import classification_report, confusion_matrix
print(classification_report(Y_test,Y_pred))
print(confusion_matrix(Y_test,Y_pred))

             precision    recall  f1-score   support

          1       0.83      0.79      0.81       233
          2       0.95      0.92      0.94       253
          3       0.97      0.98      0.98       249
          4       0.96      0.98      0.97       240
          5       0.99      0.97      0.98       236
          6       0.96      0.96      0.96       240
          7       0.75      0.95      0.84       261
          8       0.98      0.96      0.97       269
          9       0.97      0.98      0.98       284
         10       1.00      1.00      1.00       248
         11       1.00      1.00      1.00       231
         12       0.97      0.96      0.97       233
         13       0.93      0.98      0.95       244
         14       0.98      0.93      0.95       256
         15       0.98      0.94      0.96       246
         16       0.99      1.00      1.00       252
         17       0.88      0.89      0.88       249
         18       0.92      0.87      0.90   